In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pathlib import Path
from pprint import pprint
import numpy as np

import configparser

import matplotlib.pyplot as plt 
import seaborn as sns

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

import sys
sys.path.append("../")
from _helpers import *

In [2]:
# Load config
root = Path('../../')
Config = configparser.ConfigParser()
Config.read(str(root / 'config.cnf'))

queries = root / Config.get('input_files', 'queries')
queries = load_queries(str(queries))

tweets = root / Config.get('output_files', 'tweets')
twitter_urls = root / Config.get('output_files', 'twitter_urls')
altmetric_urls = root / Config.get('output_files',  'altmetric_urls')

twitter_urls = load_urls(twitter_urls)
altmetric_urls = load_altmetric(altmetric_urls)

/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [23]:
df = altmetric_urls[(altmetric_urls['clean_url'].notnull()) & (altmetric_urls.relevant == True)]
am_urls_per_source = df.groupby("venue_short")['clean_url'].apply(lambda x: set(x.unique()))

df = twitter_urls[(twitter_urls['cleaned_url'].notnull()) & (twitter_urls.relevant == True)]
tw_urls_per_source = df.groupby("venue")['cleaned_url'].apply(lambda x: set(x.unique()))

In [26]:
df = pd.DataFrame(columns=['Twitter', 'Altmetric', 'both'])
for ix, tw_urls in tw_urls_per_source.iteritems():
    try:
        tw = len(tw_urls)
        if ix in am_urls_per_source:
            am_urls = am_urls_per_source[ix]
            am = len(am_urls)
            both = len(tw_urls.intersection(am_urls))
        else:
            am = 0
            both = 0
        df.loc[ix] = [tw, am, both]
    except:
        raise
df.sort_values(by="both")
# df.index.name = "venue"
# df.to_csv("results.csv")

,Twitter,Altmetric,both
slate,219,53,1
wired,255,1,1
bostonglobe,226,4,3
sfchronicle,325,19,5
latimes,2065,10,10
theglobeandmail,176,24,23
foxnews,3474,253,231
washingtonpost,4617,593,315
nytimes,4528,342,318
theguardian,2113,591,542
